In [1]:
import sys
!{sys.executable} -m pip install evodiff

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
torch.cuda.is_available()

True

In [11]:
pip install --upgrade numpy scipy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 11.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mdanalysis 2.7.0 requires numpy<2.0,>=1.22.3, but you have numpy 2.1.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Conditional generation


### Evolutionary guided sequence generation with EvoDiff

In [4]:
from evodiff.pretrained import OA_DM_640M

checkpoint = OA_DM_640M()
model, collater, tokenizer, scheme = checkpoint

# Move the model to CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [5]:
model

ByteNetLMTime(
  (embedder): ByteNetTime(
    (time_encoding): PositionalEncoding1D()
    (embedder): Embedding(31, 8, padding_idx=28)
    (up_embedder): PositionFeedForward(
      (conv): Conv1d(8, 1280, kernel_size=(1,), stride=(1,))
    )
    (layers): ModuleList(
      (0): ByteNetBlock(
        (conv): MaskedConv1d(1280, 1280, kernel_size=(5,), stride=(1,), padding=(2,))
        (sequence1): Sequential(
          (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (1): GELU(approximate='none')
          (2): PositionFeedForward(
            (conv): Conv1d(1280, 1280, kernel_size=(1,), stride=(1,))
          )
          (3): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (4): GELU(approximate='none')
        )
        (sequence2): Sequential(
          (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (1): GELU(approximate='none')
          (2): PositionFeedForward(
            (conv): Conv1d(1280, 1280, kernel_size=(1,), stri

In [18]:
pip install torch_geometric torch_scatter

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3654726 sha256=a58e41fac481c322914e4e2879ed3dfd302a37b87c13015578fe3f0647fe68cf
  Stored in directory: /home/johnny/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch_scatter
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install biotite==0.35.0

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.0/32.0 MB 6.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: biotite
    Found existing installation: biotite 1.0.1
    Uninstalling biotite-1.0.1:
      Successfully uninstalled biotite-1.0.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
from evodiff.conditional_generation import inpaint_simple

sequence = 'DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI'
start_idx = 1
end_idx = 10


sample, entire_sequence, generated_idr = inpaint_simple(model, sequence,
                                                        start_idx, end_idx,
                                                        tokenizer=tokenizer,
                                                        device='cuda')

print("original sequence:", sequence)
print("generated sequence", entire_sequence)


print("\noriginal region:", sequence[start_idx:end_idx])
print("generated region:", generated_idr)

/home/johnny/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 9/9 [00:00<00:00, 25.16it/s]

original sequence: DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI
generated sequence DALQHALGPGEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI

original region: QTERTVRSF
generated region: ALQHALGPG


## Condition on indexes (input list that masks the amino acids)

In [9]:
# from evodiff.pretrained import OA_DM_38M
# from evodiff.conditional_generation import inpaint_simple

# # Load the model
# checkpoint = OA_DM_38M()
# model, collater, tokenizer, scheme = checkpoint

# # Move the model to CUDA if available
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = model.to(device)


def generate_conditional_sequence_OA_DM(sequence: str, condition_indices: list)-> str:
    # Convert sequence to a list for mutability
    masked_sequence = list(sequence)
    
    # Mask specified indices with '#'
    for idx in condition_indices:
        masked_sequence[idx] = '#'  # Mark the position for inpainting
    
    # Join to form the masked sequence
    masked_sequence = ''.join(masked_sequence)
    generated_sequence = masked_sequence
    
    # Sequentially apply inpainting to each index individually
    for idx in condition_indices:
        # Apply inpainting to the masked character position
        sample, entire_sequence, generated_region = inpaint_simple(
            model, generated_sequence, idx, idx + 1, tokenizer=tokenizer, device=device
        )
        # Replace the masked character with the generated content
        generated_sequence = generated_sequence[:idx] + generated_region + generated_sequence[idx + 1:]
    
    return generated_sequence

# Example
sequence = 'DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI'
condition_indices = [1, 4, 6, 8, 10]

generated_sequence = generate_conditional_sequence_OA_DM(sequence, condition_indices)
print(f"Original sequence:\t{sequence}")
print(f"Generated sequence:\t{generated_sequence}")

100%|██████████| 1/1 [00:00<00:00, 52.72it/s]

Original sequence:	DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI
Generated sequence:	DITESTVRIFAGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI
